In [ ]:
%reload_ext autoreload
%autoreload 2

from alpharaw.mzml import MzMLReader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from alphabase.spectral_library.base import SpecLibBase

In [ ]:
base_folder = "/Users/georgwallmann/Documents/data/alphadia-validate/"
data_folder = f'{base_folder}/data'
output_folder = f'{base_folder}/output'

#### Obtain raw & results data
Note: this is just required if you did not run the search yourself in search_1.10.0.ipynb

In [ ]:
for folder in [data_folder, output_folder]:
    if not os.path.exists(folder):
        os.makedirs(folder)

# HeLa library as used in the getting started guide
library_url = "https://datashare.biochem.mpg.de/s/Nsp8CaHMBf7FHq1/download?path=%2Ffirst_pass&files=speclib.hdf"

# Bulk injections of HeLa cell lysate acquired on the Orbitrap Astral
raw_data_url_list = [
    "https://datashare.biochem.mpg.de/s/Nsp8CaHMBf7FHq1/download?path=%2Fraw_data&files=20231023_OA3_TiHe_ADIAMA_HeLa_200ng_Evo01_21min_F-40_iO_after_01.raw",
    #"https://datashare.biochem.mpg.de/s/Nsp8CaHMBf7FHq1/download?path=%2Fraw_data&files=20231023_OA3_TiHe_ADIAMA_HeLa_200ng_Evo01_21min_F-40_iO_after_02.raw",
    #"https://datashare.biochem.mpg.de/s/Nsp8CaHMBf7FHq1/download?path=%2Fraw_data&files=20231023_OA3_TiHe_ADIAMA_HeLa_200ng_Evo01_21min_F-40_iO_after_02.raw",
]

precursors_tsv_url = "https://datashare.biochem.mpg.de/s/9JIfPb4b0737qj9/download?path=%2Fguide_results_libfree-gui%2Fsecond_pass&files=precursors.tsv"

speclib_url = "https://datashare.biochem.mpg.de/s/9JIfPb4b0737qj9/download?path=%2Fguide_results_libfree-gui%2Fsecond_pass&files=speclib.hdf"


from alphadia.test_data_downloader import DataShareDownloader
_ = [DataShareDownloader(url, data_folder).download() for url in raw_data_url_list]
_ = DataShareDownloader(library_url, data_folder).download()
_ = DataShareDownloader(precursors_tsv_url, output_folder).download()
_ = DataShareDownloader(speclib_url, output_folder).download()

### 1. Set up data
First we will point our notebook to the raw file and search results obtained using this raw file.

We will then load three objects into the notebook:
- The raw DIA data `dia_data`
- The search results `precursor_df`
- The spectral library `spectral_library`

In [ ]:
raw_files = [
     f"{data_folder}/20231017_OA2_TiHe_ADIAMA_HeLa_200ng_Evo011_21min_F-40_05.raw"
]

current_raw_name = os.path.basename(raw_files[0]).replace('.raw', '')
current_raw_path = raw_files[0]

search_results = output_folder

In [ ]:
precursor_df = pd.read_csv(os.path.join(search_results,'precursors.tsv'), sep='\t')
precursor_df = precursor_df[precursor_df['run'] == current_raw_name]

spectral_library = SpecLibBase()
spectral_library.load_hdf(os.path.join(search_results,'speclib.hdf'))

In [ ]:
from alphadia.data.alpharaw_wrapper import Thermo
dia_data =  Thermo(current_raw_path)

### 2. Inspect data
Next we want to inspect the data structures to get a better understanding of the data.

#### 2.1 Spectral library
We will start with the spectral library.
It's an alphabase SpecLibBase object, which contains the following attributes:
- `precursor_df`: a pandas DataFrame containing the precursor information
- `fragment_mz_df`: a pandas DataFrame containing the fragment m/z values
- `fragment_intensity_df`: a pandas DataFrame containing the fragment intensities

The `precursor_df` links to the fragment_mz_df and fragment_intensity_df via the `frag_start_idx` and `frag_stop_idx` columns.
For unique indexing we will use the `mod_seq_charge_hash` column.

In [ ]:
spectral_library.precursor_df[['precursor_mz', 'sequence', 'mods', 'mod_sites', 'charge', 'mod_seq_charge_hash', 'frag_start_idx', 'frag_stop_idx']].head()

In [ ]:
spectral_library.fragment_mz_df.head()

In [ ]:
spectral_library.fragment_intensity_df.head()

#### 2.2 Precursor data

The identified precursors following search are stored in the `precursor_df` DataFrame.
The precursors in this dataframe come from the spectral library but have aditional information on their identification.

The most important columns are:
- `mod_seq_charge_hash`: the hash of the precursor sequence and charge

The scans where they were identified called frames.
- `frame_start`: the frame number of the first frame in the run.
- `frame_stop`: the frame number of the last frame in the run

Furthermore there is the q-value and a multitude of scores that were used to identify the precursor.

In [ ]:
precursor_df.head()

#### 2.3 DIA data

Last, we have the raw DIA data objecrt loaded from the Thermo raw file.

This object contains all scans in the `dia_data.spectrum_df` DataFrame.
Each spectrum points to a collection of peak based on the `peak_start_idx` and `peak_stop_idx` columns.

These point to the `dia_data.peak_df` DataFrame, which contains the peak information.

In [ ]:
dia_data.spectrum_df.head()

In [ ]:
dia_data.peak_df.head()

#### 3 Map precursor hit from search results to raw data

Using this information we can map the identified precursors to the raw data.

We will use the `get_library_entry_by_hash` function to get the library entry for a given hash.

This function returns the library entry, the fragment m/z values and the fragment intensities.



In [ ]:
hash = precursor_df['mod_seq_charge_hash'].iloc[0]

def get_library_entry_by_hash(speclib, hash, min_intensity=0.01):
    speclib_entry = speclib.precursor_df[speclib.precursor_df['mod_seq_charge_hash'] == hash].iloc[0]

    fragment_mz = speclib.fragment_mz_df.iloc[speclib_entry.frag_start_idx:speclib_entry.frag_stop_idx].to_numpy().flatten()
    fragment_intensity = speclib.fragment_intensity_df.iloc[speclib_entry.frag_start_idx:speclib_entry.frag_stop_idx].to_numpy().flatten()
    fragment_mask = fragment_intensity > min_intensity

    fragment_mz = fragment_mz[fragment_mask]
    fragment_intensity = fragment_intensity[fragment_mask]

    # sort both by mz
    fragment_order = np.argsort(fragment_mz)
    fragment_mz = fragment_mz[fragment_order]
    fragment_intensity = fragment_intensity[fragment_order]

    return speclib_entry, fragment_mz, fragment_intensity

speclib_entry, mz_library, intensity_library = get_library_entry_by_hash(spectral_library, hash)
precursor_entry = precursor_df[precursor_df['mod_seq_charge_hash'] == hash].iloc[0]

In [ ]:
jit_data = dia_data.jitclass()

precursor_query = np.array([[speclib_entry.precursor_mz, speclib_entry.precursor_mz]], dtype=np.float32)
scan_limits = np.array([[precursor_entry.scan_start, precursor_entry.scan_stop, 1]], dtype=np.int64)
frame_limits = np.array([[precursor_entry.frame_start, precursor_entry.frame_stop, 1]], dtype=np.int64)

dense, precursor_index = jit_data.get_dense(
    frame_limits,
    scan_limits,
    mz_library,
    30,
    precursor_query,
)

#### 4 Visualize precursor data

Now, we want to viosualize the retrieved spectrum data.
We will start by visualizing the observed spectrum and the library spectrum.

The spectrum data `dense` is a 5 dimensional numpy array with a dense slice of the spectrum space.
The dimensions are:
- 0: either intensity information 0 or relative mass error 1
- 1: index of the fragment mz which was queried
- 2: ion mobility dimension (will be zero for DIA data)
- 3: The observations in the DIA cycle. As there might be multiple quadrupole windows where the precursor was detected, this will be a list of observations.
- 4: Retention time datapoints.

First we will select the intensity dimension and sum over all other dimensions but the fragment mz dimension.


In [ ]:
intensity_observed = dense[0].sum(axis=(1,2,3))
intensity_observed_normalized = intensity_observed / intensity_observed.max()
intensity_library_normalized = intensity_library / intensity_library.max()

plt.stem(mz_library, intensity_observed_normalized)
plt.stem(mz_library, -intensity_library_normalized)
plt.show()

Finally, we will visualize the Precusor ion chromatogram.
We will again select the intensity dimension and sum over ion mobility and observation but leave the retention time dimension.


In [ ]:
xic_observed = dense[0].sum(axis=(1,2))
for i in range(xic_observed.shape[0]):
    plt.plot(xic_observed[i])
plt.show()